<a href="https://colab.research.google.com/github/ankit-rathi/AR-Talks/blob/master/Nifty100_Buy_Sell_Signals_New.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install yfinance
!pip install talib-binary

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 62 kB 1.3 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement talib-binary (from versions: none)
ERROR: No matching distribution found for talib-binary


In [2]:
import pandas as pd
import numpy as np

mypf = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Data-Science-with-Python/master/Stock_Analysis/data/Nifty100Scrips.csv')

mypf = mypf[mypf['InScope'] == 1]

stock_n100 = mypf['Symbol'].unique()

exclude = ['CADILAHC','MMTC']

stock_n100 = mypf[~mypf['Symbol'].isin(exclude) ]['Symbol']

#stock_ids

stock_ids = stock_n100

In [3]:
# import necessary libraries 

%matplotlib inline
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from tabulate import tabulate
import warnings
warnings.filterwarnings('ignore')
import pandas_datareader.data as web
import yfinance as yfin
import talib 
yfin.pdr_override()

ModuleNotFoundError: ignored

In [ ]:
def MovingAverageCrossStrategy(stock_symbol = 'ULTRACEMCO.NS', start_date = '2018-01-01', end_date = '2020-01-01', 
                               short_window = 20, mid_window = 50, long_window = 200, moving_avg = 'SMA', display_table = True):
    '''
    The function takes the stock symbol, time-duration of analysis, 
    look-back periods and the moving-average type(SMA or EMA) as input 
    and returns the respective MA Crossover chart along with the buy/sell signals for the given period.
    '''
    # stock_symbol - (str)stock ticker as on Yahoo finance. Eg: 'ULTRACEMCO.NS' 
    # start_date - (str)start analysis from this date (format: 'YYYY-MM-DD') Eg: '2018-01-01'
    # end_date - (str)end analysis on this date (format: 'YYYY-MM-DD') Eg: '2020-01-01'
    # short_window - (int)lookback period for short-term moving average. Eg: 5, 10, 20 
    # long_window - (int)lookback period for long-term moving average. Eg: 50, 100, 200
    # moving_avg - (str)the type of moving average to use ('SMA' or 'EMA')
    # display_table - (bool)whether to display the date and price table at buy/sell positions(True/False)

    calc_date = (datetime.datetime.strptime(start_date, '%Y-%m-%d') + datetime.timedelta(days=-200)).strftime('%Y-%m-%d')

    # import the closing price data of the stock for the aforementioned period of time in Pandas dataframe
    start = datetime.datetime(*map(int, calc_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-'))) 
    stock_df = web.DataReader(stock_symbol,  data_source='yahoo', start = start, end = end)['Close']
    stock_df = pd.DataFrame(stock_df) # convert Series object to dataframe 
    stock_df.columns = {'Close Price'} # assign new colun name
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows 
                        
    # column names for long and short moving average columns
    short_window_col = str(short_window) + '_' + moving_avg
    mid_window_col = str(mid_window) + '_' + moving_avg
    long_window_col = str(long_window) + '_' + moving_avg  
  
    if moving_avg == 'SMA':
        # Create a short simple moving average column
        stock_df[short_window_col] = stock_df['Close Price'].rolling(window = short_window, min_periods = 1).mean()
        
        stock_df[mid_window_col] = stock_df['Close Price'].rolling(window = mid_window, min_periods = 1).mean()

        # Create a long simple moving average column
        stock_df[long_window_col] = stock_df['Close Price'].rolling(window = long_window, min_periods = 1).mean()

    elif moving_avg == 'EMA':
        # Create short exponential moving average column
        stock_df[short_window_col] = stock_df['Close Price'].ewm(span = short_window, adjust = False).mean()
        
        stock_df[mid_window_col] = stock_df['Close Price'].ewm(span = mid_window, adjust = False).mean()

        # Create a long exponential moving average column
        stock_df[long_window_col] = stock_df['Close Price'].ewm(span = long_window, adjust = False).mean()

    # create a new column 'Signal' such that if faster moving average is greater than slower moving average 
    # then set Signal as 1 else 0.
    stock_df['Signal'] = 0.0  
    stock_df['Signal'] = np.where(((stock_df[long_window_col] > stock_df[mid_window_col]) & (stock_df['Close Price'] > stock_df[mid_window_col]))
                                , 1.0, stock_df['Signal']) 
    
    stock_df['Signal'] = np.where(((stock_df[long_window_col] < stock_df[mid_window_col]) & (stock_df['Close Price'] < stock_df[mid_window_col]))
                                  , -1.0, stock_df['Signal']) 

    # create a new column 'Position' which is a day-to-day difference of the 'Signal' column. 
    stock_df['Position'] = stock_df['Signal'].diff()

    stock_df = stock_df[stock_df.index >= start_date]

    # plot close price, short-term and long-term moving averages
    plt.figure(figsize = (20,10))
    plt.tick_params(axis = 'both', labelsize = 14)
    stock_df['Close Price'].plot(color = 'k', lw = 1, label = 'Close Price')  
    #stock_df[short_window_col].plot(color = 'g', lw = 1, label = short_window_col)
    stock_df[mid_window_col].plot(color = 'm', lw = 1, label = mid_window_col)
    stock_df[long_window_col].plot(color = 'b', lw = 1, label = long_window_col) 
    
    # plot 'buy' signals
    plt.plot(stock_df[stock_df['Position'] >= 1].index, 
            stock_df['Close Price'][stock_df['Position'] >= 1], 
            '^', markersize = 15, color = 'g', alpha = 0.7, label = 'buy')

    # plot 'sell' signals
    plt.plot(stock_df[stock_df['Position'] <= -1].index, 
            stock_df['Close Price'][stock_df['Position'] <= -1], 
            'v', markersize = 15, color = 'r', alpha = 0.7, label = 'sell')
    
    plt.ylabel('Price in ₹', fontsize = 16 )
    plt.xlabel('Date', fontsize = 16 )
    plt.title(str(stock_symbol) + ' - ' + str(moving_avg) + ' Crossover', fontsize = 20)
    plt.legend()
    plt.grid()
    plt.show()
    
    if display_table == True:
        print(stock_symbol)
        df_pos = stock_df[(stock_df['Position'] >= 1) | (stock_df['Position'] <= -1)]
        df_pos['Position'] = df_pos['Position'].apply(lambda x: 'Buy' if x >= 1 else 'Sell')
        df_pos = df_pos[df_pos['Signal'] != 0]
        df_pos['Prev_Signal'] = df_pos['Signal'].shift(1)
        df_pos = df_pos[df_pos['Signal'] != df_pos['Prev_Signal']]
        df_pos.drop('Prev_Signal', axis=1, inplace=True)
        df_pos.drop(short_window_col, axis=1, inplace=True)
        print(tabulate(df_pos, headers = 'keys', tablefmt = 'psql'))
        
def plot_buy_sell(stock_id, start_date, table):
    
    #start_date = '2011-01-01'
    #end_date = '2020-03-01'    
    end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
    strategy = 'SMA'
    short_window = 20
    mid_window = 50
    long_window = 200
    MovingAverageCrossStrategy(stock_id, start_date, end_date, short_window, mid_window, long_window, strategy, display_table = table)
 

In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

In [ ]:
start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
stock_id = '^NSEI'
plot_buy_sell(stock_id, start_date, True)

In [ ]:
def get_buy_sell_signal(stock_symbol, start_date, end_date, moving_avg, short_window, mid_window,long_window):

    stock_df = web.DataReader(stock_symbol,  data_source='yahoo', start = start_date, end = end_date)['Close']
    stock_df = pd.DataFrame(stock_df) # convert Series object to dataframe 
    stock_df.columns = {'Close Price'} # assign new colun name
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows 

    # column names for long and short moving average columns
    short_window_col = str(short_window) + '_' + moving_avg
    mid_window_col = str(mid_window) + '_' + moving_avg
    long_window_col = str(long_window) + '_' + moving_avg  

    if moving_avg == 'SMA':
        # Create a short simple moving average column
        stock_df[short_window_col] = stock_df['Close Price'].rolling(window = short_window, min_periods = 1).mean()
        
        stock_df[mid_window_col] = stock_df['Close Price'].rolling(window = mid_window, min_periods = 1).mean()

        # Create a long simple moving average column
        stock_df[long_window_col] = stock_df['Close Price'].rolling(window = long_window, min_periods = 1).mean()

    elif moving_avg == 'EMA':
        # Create short exponential moving average column
        stock_df[short_window_col] = stock_df['Close Price'].ewm(span = short_window, adjust = False).mean()
        
        stock_df[mid_window_col] = stock_df['Close Price'].rolling(window = mid_window, min_periods = 1).mean()

        # Create a long exponential moving average column
        stock_df[long_window_col] = stock_df['Close Price'].ewm(span = long_window, adjust = False).mean()

    # create a new column 'Signal' such that if faster moving average is greater than slower moving average 
    # then set Signal as 1 else 0.
    stock_df['Signal'] = 0.0  
    stock_df['Signal'] = np.where(((stock_df[long_window_col] > stock_df[mid_window_col]) & (stock_df['Close Price'] > stock_df[mid_window_col]))
                                , 1.0, stock_df['Signal']) 
    
    stock_df['Signal'] = np.where(((stock_df[long_window_col] < stock_df[mid_window_col]) & (stock_df['Close Price'] < stock_df[mid_window_col]))
                                  , -1.0, stock_df['Signal']) 

    # create a new column 'Position' which is a day-to-day difference of the 'Signal' column. 
    stock_df['Position'] = stock_df['Signal'].diff()
    
    df_pos = stock_df[(stock_df['Position'] >= 1) | (stock_df['Position'] <= -1)]
    df_pos['Position'] = df_pos['Position'].apply(lambda x: 'Buy' if x >= 1 else 'Sell')
    df_pos = df_pos[df_pos['Signal'] != 0]
    df_pos['Prev_Signal'] = df_pos['Signal'].shift(1)
    df_pos = df_pos[df_pos['Signal'] != df_pos['Prev_Signal']]
    df_pos.drop('Prev_Signal', axis=1, inplace=True)
    df_pos.drop(short_window_col, axis=1, inplace=True)
        
    df_pos['Stock'] = stock_symbol
    return df_pos.tail(1)

In [ ]:
start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
moving_avg = 'SMA'
short_window = 20
mid_window = 50
long_window = 200

df_buy_sell = pd.DataFrame()

for stock_id in stock_ids:
    stock_id = stock_id.upper() + '.NS'
    tmp = get_buy_sell_signal(stock_id, start_date, end_date, moving_avg, short_window, mid_window, long_window)
    df_buy_sell = df_buy_sell.append(tmp)
    
df_buy_sell.sort_index()

In [ ]:
df_buy_sell[df_buy_sell['Position'] == 'Buy'].sort_index().tail(20)

In [ ]:
df_buy_sell[df_buy_sell['Position'] == 'Sell'].sort_index().tail(20)

In [ ]:
start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
stock_id = 'IRCTC.NS'
plot_buy_sell(stock_id, start_date, True)